In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import math

In [ ]:
%matplotlib 



# these values will be read from Binner
bits_per_bin = 8
sort_codes = 1
brightness = 1
###
numNeurons = 16
numNeuronsSlice = (numNeurons * 10) + 1
mat = sio.loadmat(r'C:\Users\Diya\Documents\MATLAB\H_matrix.mat')
H_matrix = mat['H_matrix']
H_matrix = H_matrix[0:numNeuronsSlice, :]
###
input_data = np.empty((1, numNeurons))
predicSignal = np.empty((1,5))


def count_mapper(values):

    byte_packet = np.array(values, dtype='<u4').view(np.uint8)
    
    # convert to bits
    bits_packet = np.unpackbits(byte_packet, bitorder='little')
 
    # organize by bits_per_bin
    shaped_bits = np.reshape(bits_packet, (-1, bits_per_bin))

    # make our own padding due to packbits not handling endian padding
    if shaped_bits.shape[1] < 8:
        padding = np.zeros((shaped_bits.shape[0], 8 - bits_per_bin), dtype=np.uint8)    
        shaped_bits = np.hstack((shaped_bits, padding))
    
    # compress back into sort codes
    counts = np.packbits(shaped_bits, bitorder='little').astype(np.uint32)
    result = np.reshape(counts, (numNeurons, 1))
    return result.T

def DuplicateAndShift(spikes,numlags):
    numpts, Nin = np.shape(spikes);
    DS_spikes = np.zeros((numpts,Nin*numlags));
    

    for i in range(Nin*numlags):
        n = math.ceil((i+1)/numlags);
        lag = i%numlags;
        prepend = np.zeros((lag+1));
        spike_slice = spikes[0:(-1-lag), n-1]
        DS_spikes[:,i] = np.hstack((prepend, spike_slice));

    return DS_spikes

class Always:   #StateID = 0

    def s_Mode_recprev():
        p_State.setTimeout(2, GetGlobals)

    def s_Mode_idle():
        global predicSignal
        print(predicSignal)
        plt.close('all')

    def s_Brightness_change(value):
        global brightness
        brightness = value


class GetGlobals:   #StateID = 1

    def s_State_enter():
        
        # get bits per bin and number of sort codes from Binner
        global bits_per_bin
        bits_per_bin = int(syn.getParameterValue('Binner1','NumBits'))
        global sort_codes
        sort_codes = int(syn.getParameterValue('Binner1','NumSorts'))
        #print('bits per bin', bits_per_bin, 'sort codes', sort_codes)
        
        p_State.setTimeout(.1, StartCounting)


class StartCounting:   #StateID = 2

    def s_Input1_rise():    # when Binner1 strobes
        global input_data
        global predicSignal
        # read binned sort code counts
        values = syn.getParameterValues('Binner1','out_Main')

        # expand sort code counts
        counts = count_mapper(values)
        input_data = np.concatenate((input_data, counts))
        if np.size(input_data, 0) > 10:
            duplicated = DuplicateAndShift(input_data, 10)
            size_array = np.size(duplicated, 0)
            bias = np.ones((size_array, 1))
            concat = np.hstack((bias, duplicated))
            output = np.dot(concat, H_matrix)
            predicSignal = np.vstack((predicSignal, output[-1, :]))
            print(predicSignal[-1,:])

        # do something with the sorts/channels
        #ChanTotal = sum(counts)
        #GrandTotal = sum(ChanTotal)
        #print(GrandTotal)
        
        #p_Param.AmpA_write(GrandTotal)
        #p_Output.StimTrig.fire()
        #print('1 count')
        #print(input_data)


        # plot heat map
        #plt.cla()
        #plt.imshow(
                    #brightness * counts.T,
                    #cmap='gray',
                    #vmin=0,
                    #vmax=2**(bits_per_bin-1),
                    #extent=[1,sort_codes,counts.shape[1],1],
                    #aspect='auto'
                    #)
        #plt.xticks(np.arange(1, sort_codes+1))
        #plt.yticks(np.arange(1, counts.shape[1]+1))
        #plt.ylabel('channel')
        #plt.xlabel('sort code')
        #plt.show()



In [2]:
array = np.array([[0,0,0,0,0]])
print(array)
print(array.shape)

[[0 0 0 0 0]]
(1, 5)


In [3]:
array1 = np.array([[0,0,0,0,0,0]])
array2 = np.array([[1,2,3,4,5,6]])
array3 = np.concatenate((array1,array2))
print(array3)

[[0 0 0 0 0 0]
 [1 2 3 4 5 6]]


In [4]:
dummyarray = np.zeros((10,32))
print(dummyarray)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 

In [8]:
insertion1 = np.array([[1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0]])
insertion2 = np.array([[0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,2,2]])
insertion3 = np.array([[0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0]])
insertion4 = np.array([[0,4,0,0,0,4,0,0,0,0,0,0,0,4,0,0,0,4,0,0,0,0,0,4,0,0,0,0,0,0,4,0]])
insertion5 = np.array([[5,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,5,0,0,0,0,0,5,5,5,0,0,0]])
print(insertion1.shape)

(1, 32)


In [6]:
dummyarray = np.roll(dummyarray,-1,axis = 0)
dummyarray[-1,:] = insertion1
print(dummyarray)
dummyarray = np.roll(dummyarray,-1,axis = 0)
dummyarray[-1,:] = insertion2
print(dummyarray)
dummyarray = np.roll(dummyarray,-1,axis = 0)
dummyarray[-1,:] = insertion3
print(dummyarray)
dummyarray = np.roll(dummyarray,-1,axis = 0)
dummyarray[-1,:] = insertion4
print(dummyarray)
dummyarray = np.roll(dummyarray,-1,axis = 0)
dummyarray[-1,:] = insertion5
print(dummyarray)


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 1. 

In [8]:
a = np.roll(a,-1,axis = 0)
a[-1,:] = 0

NameError: name 'a' is not defined

In [9]:
if np.size(input_data, 0) > 10:
            duplicated = DuplicateAndShift(input_data, 10)
            size_array = np.size(duplicated, 0)
            bias = np.ones((size_array, 1))
            concat = np.hstack((bias, duplicated))
            output = np.dot(concat, H_matrix)
            predicSignal = np.vstack((predicSignal, output[-1, :]))
            print(predicSignal[-1,:])

NameError: name 'input_data' is not defined

In [10]:
def DuplicateAndShift(spikes,numlags):
    numpts, Nin = np.shape(spikes);
    DS_spikes = np.zeros((numpts,Nin*numlags));
    

    for i in range(Nin*numlags):
        n = math.ceil((i+1)/numlags);
        lag = i%numlags;
        prepend = np.zeros((lag+1));
        spike_slice = spikes[0:(-1-lag), n-1]
        DS_spikes[:,i] = np.hstack((prepend, spike_slice));

    return DS_spikes

In [11]:
duplicated = DuplicateAndShift(dummyarray, 10)

In [12]:
print(dummyarray.shape)
print(duplicated.shape)

(10, 32)
(10, 320)


In [13]:
size_array = np.size(duplicated, 0)
bias = np.ones((size_array, 1))
concat = np.hstack((bias, duplicated))

print(concat.shape)

(10, 321)
